#Silver Layer - Transformação de Vendas: Detail

In [0]:
%run ./00_Setup_Environment

In [0]:
df_detail = spark.table("bronze.sales_order_detail")
df_order = spark.table("bronze.sales_order_header")

df_products = spark.table("silver.product")

In [0]:
# Agrupar endereços por cliente
'''df_sales_detail_grouped = df_detail.alias("d") \
    .join(df_order.alias("h"),
          col("d.SalesOrderID") == col("h.SalesOrderID"), 
          "left") \
    .groupBy("d.SalesOrderID") \
    .agg(count("d.SalesOrderDetailID").alias("total_procucts"))

df_sales_detail_grouped.sort('SalesOrderID').display()'''

In [0]:
df_detail = df_detail.withColumn("OrderQty", col("OrderQty").cast("int"))
df_detail = df_detail.withColumn("UnitPrice", regexp_replace(col("UnitPrice"), ",", ".").cast("decimal(18,2)"))
df_detail = df_detail.withColumn("UnitPriceDiscount", regexp_replace(col("UnitPriceDiscount"), ",", ".").cast("decimal(18,2)"))

In [0]:
df_silver_detail = df_detail.alias("d") \
    .join(df_products.alias("p"), 
          col("d.ProductID") == col("p.product_id"), 
          "left") \
    .select(
            #Identificadores
            col("d.SalesOrderDetailID").alias("detail_id"),
            col("d.SalesOrderID").alias("order_id"),
            col("d.ProductID").alias("product_id"),
            
            # Informações do produto (enriquecimento)
            col("p.product_name"),
            col("p.category_name"),
            col("p.model_name"),
            
            # Quantidades e preços
            col("d.OrderQty").alias("quantity"),
            col("d.UnitPrice").cast("decimal(18,2)").alias("unit_price"),
            col("d.UnitPriceDiscount").cast("decimal(18,2)").alias("discount_percent"),
            
            # Cálculos financeiros
            round(col("d.OrderQty") * col("d.UnitPrice"), 2).alias("subtotal"),
            round(col("d.OrderQty") * col("d.UnitPrice") * col("d.UnitPriceDiscount"), 2).alias("discount_amount"),
            round(col("d.OrderQty") * col("d.UnitPrice") * (1 - col("d.UnitPriceDiscount")), 2).alias("line_total"),

            # Metadados
            col("d.ModifiedDate").alias("detail_modified_date"),
            current_timestamp().alias("processed_timestamp"),
            
            # Flags de qualidade
            when(col("d.OrderQty") > 0, True).otherwise(False).alias("valid_quantity"),
            when(col("d.UnitPrice") >= 0, True).otherwise(False).alias("valid_price")
        )

In [0]:
df_silver_detail.display()

In [0]:
path = f"{silver_path}/sales_detail"
df_silver_detail.write \
    .mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .save(path)

spark.sql("USE adventureworks.silver")
df_silver_detail.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("sales_detail")
                                                     
count = df_silver_detail.count()
log_etl("sales", "silver", "SUCCESS", count)

print(f"Sales Detail: {count} registros")